In [8]:
# 0907/2024년 지역축제 개최계획(수정).csv

import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('/Users/bokyung/Desktop/LLM-RAG-LANGCHAIN/0907/2024년 지역축제 개최계획(수정).csv', encoding='utf-8')

# 첫번째 row를 header로 설정
df.columns = df.iloc[0]
df = df[1:]

# 데이터 확인
print(df.head())

0 연번 광역자치\n단체명 기초자치\n단체명          축제명  축제유형       개최기간                 개최장소  \
1  1        서울         -     서울드럼페스티벌  문화예술  5.25~5.26                  노들섬   
2  2        서울         -     서울뮤직페스티벌  문화예술  9.27~9.29                  노들섬   
3  3        서울         -       서울국악축제  전통역사       9월 중                   미정   
4  4        서울         -   서울페스타 2024  문화예술  5.1.~5.6.  서울광장, 광화문광장 등 서울 전역   
5  5        서울         -  서울 바비큐 페스티벌    기타  5.4.~5.5.                 노을공원   

0 개최방식 최초\n개최년도(년) 합계 예산(백만원)  ... 방문객수(2023년) 외국인(명)      전담조직명  조직형태  \
1   대면        1999       564   ...                미집계  서울시 문화예술과   지자체   
2   대면        2019     1,062   ...               500   서울시 문화예술과   지자체   
3   대면        2019       207   ...                미집계  서울시 문화예술과   지자체   
4   대면        2022     2,115   ...           242,330      서울관광재단  재단법인   
5   대면        2024       350   ...                 0      서울관광재단  재단법인   

0                         설립근거 국비 지원 부처명 축제 담당자 기관명 축제 담당자 부서명 축제 담당자 직급/직책  \
1

In [14]:
import pandas as pd
from typing import List
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

# OpenAI 클라이언트 설정
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# SentenceTransformer 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# ChromaDB 클라이언트 설정
chroma_client = chromadb.Client(Settings(persist_directory="./chroma_db"))

# 컬렉션 생성 또는 기존 컬렉션 가져오기
collection = chroma_client.get_or_create_collection(name="festival_collection")

# CSV 파일 읽기
df = pd.read_csv('/Users/bokyung/Desktop/LLM-RAG-LANGCHAIN/0907/2024년 지역축제 개최계획(수정).csv', encoding='utf-8')
df.columns = df.iloc[0]
df = df[1:]

# 열 이름에서 개행 문자 제거 및 공백 제거
df.columns = df.columns.str.replace('\n', '').str.strip()

def prepare_documents(df: pd.DataFrame):
    documents = []
    for _, row in df.iterrows():
        doc = f"축제명: {row['축제명']}, 지역: {row['광역자치단체명']} {row['기초자치단체명']}, "
        doc += f"유형: {row['축제유형']}, 기간: {row['개최기간']}, 장소: {row['개최장소']}, "
        doc += f"방식: {row['개최방식']}, 예산: {row['합계 예산(백만원)']}, "
        doc += f"방문객 수: {row['방문객수(2023년) 합계']}"
        documents.append(doc)
    return documents

def add_documents(texts: List[str]):
    # 문서 임베딩
    embeddings = model.encode(texts).tolist()
    
    # ChromaDB에 문서 추가
    collection.add(
        embeddings=embeddings,
        documents=texts,
        ids=[f"doc_{i}" for i in range(len(texts))]
    )

def query(question: str, k: int = 3):
    # 질문 임베딩
    query_embedding = model.encode(question).tolist()
    
    # 유사한 문서 검색
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k
    )
    
    return results['documents'][0]

def generate_answer(question: str, context: List[str]):
    # OpenAI API를 사용하여 답변 생성
    prompt = f"Context:\n{''.join(context)}\n\nQuestion: {question}\nAnswer:"
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant knowledgeable about Korean festivals."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7,
    )
    return response.choices[0].message.content.strip()

# 메인 실행 부분
if __name__ == "__main__":
    # 문서 준비 및 추가
    documents = prepare_documents(df)
    add_documents(documents)

    # 질문 및 답변 생성
    question = "서울에서 열리는 문화예술 축제는 무엇인가요?"
    relevant_docs = query(question)
    answer = generate_answer(question, relevant_docs)
    print(f"질문: {question}")
    print(f"답변: {answer}")

/Users/bokyung/Desktop/LLM-RAG-LANGCHAIN/.conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Add of existing embedding ID: doc_0
Add of existing embedding ID: doc_1
Add of existing embedding ID: doc_2
Add of existing embedding ID: doc_3
Add of existing embedding ID: doc_4
Add of existing embedding ID: doc_5
Add of existing embedding ID: doc_6
Add of existing embedding ID: doc_7
Add of existing embedding ID: doc_8
Add of existing embedding ID: doc_9
Add of existing embedding ID: doc_10
Add of existing embedding ID: doc_11
Add of existing embedding ID: doc_12
Add of existing embedding ID: doc_13
Add of existing embedding ID: doc_14
Add of existing embedding ID

질문: 서울에서 열리는 문화예술 축제는 무엇인가요?
답변: 서울에서 열리는 문화예술 축제로는 "제2회 금천 청년축제"가 있습니다. 해당 축제는 서울 금천구에서 09.28.(예정)에 열리며, 방식은 대면이고 예산은 89만원이며 방문객 수는 200명이 예상됩니다.


In [11]:
print(df.columns)

Index(['연번', '광역자치\n단체명', '기초자치\n단체명', '축제명', '축제유형', '개최기간', '개최장소', '개최방식',
       '최초\n개최년도(년)', '합계 예산(백만원)', '국비 예산(백만원)', '지방비 예산(백만원)',
       '기타\n(민간, 자부담 등) 예산(백만원)', '방문객수(2023년) 합계', '방문객수(2023년) 내국인(명)',
       '방문객수(2023년) 외국인(명)', '전담조직명', '조직형태', '설립근거', '국비 지원 부처명',
       '축제 담당자 기관명', '축제 담당자 부서명', '축제 담당자 직급/직책', '축제 담당자 성명', '축제 담당자 연락처'],
      dtype='object', name=0)


In [7]:
import os
from typing import List
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

# py 환경변수에서 키 가져오기
from dotenv import load_dotenv
load_dotenv()
# OpenAI 클라이언트 설정
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# SentenceTransformer 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# ChromaDB 클라이언트 설정
chroma_client = chromadb.Client(Settings(persist_directory="./chroma_db"))

# 컬렉션 생성 또는 기존 컬렉션 가져오기
collection = chroma_client.get_or_create_collection(name="my_collection")

def add_documents(texts: List[str], ids: List[str]):
    # 문서 임베딩
    embeddings = model.encode(texts).tolist()
    
    # ChromaDB에 문서 추가
    collection.add(
        embeddings=embeddings,
        documents=texts,
        ids=ids
    )

def query(question: str, k: int = 3):
    # 질문 임베딩
    query_embedding = model.encode(question).tolist()
    
    # 유사한 문서 검색
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k
    )
    
    return results['documents'][0]

def generate_answer(question: str, context: List[str]):
    # OpenAI API를 사용하여 답변 생성
    prompt = f"Context:\n{''.join(context)}\n\nQuestion: {question}\nAnswer:"
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7,
    )
    return response.choices[0].message.content.strip()

# 예시 사용
if __name__ == "__main__":
    # 문서 추가
    docs = [
        "파이썬은 간단하고 배우기 쉬운 프로그래밍 언어입니다.",
        "RAG는 Retrieval-Augmented Generation의 약자입니다.",
        "임베딩은 텍스트를 숫자 벡터로 변환하는 과정입니다."
    ]
    add_documents(docs, ["doc1", "doc2", "doc3"])

    # 질문 및 답변 생성
    question = "RAG란 무엇인가요?"
    relevant_docs = query(question)
    answer = generate_answer(question, relevant_docs)
    print(f"질문: {question}")
    print(f"답변: {answer}")

/Users/bokyung/Desktop/LLM-RAG-LANGCHAIN/.conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Insert of existing embedding ID: doc1
Insert of existing embedding ID: doc2
Insert of existing embedding ID: doc3
Add of existing embedding ID: doc1
Add of existing embedding ID: doc2
Add of existing embedding ID: doc3


질문: RAG란 무엇인가요?
답변: RAG는 Retrieval-Augmented Generation의 약자로, 정보 검색 및 생성을 결합하여 자연어 처리 작업을 수행하는 모델을 가리킵니다. RAG 모델은 미리 준비된 검색 엔진을 활용하여 텍스트를 검색하고, 이를 기반으로 새로운 텍스트를 생성하는 형태로 동작합니다.


In [ ]:
%pip install --upgrade numpy

In [ ]:
%pip install openai sentence-transformers

In [ ]:
%pip install numpy==1.26.4

In [ ]:
%pip install --upgrade chromadb

In [ ]:
%pip install openai sentence-transformers chromadb